## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('F:/DeepLearning/Fake_News_Classifier_Using_LSTM/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
###Drop Nan Values
df=df.dropna()

df.shape

(18285, 5)

In [6]:
df.head(60)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that...",0
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...,0


In [7]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [8]:
## Get the Dependent features
y=df['label']

In [9]:
X.shape

(18285, 4)

In [10]:
y.shape

(18285,)

In [11]:
import tensorflow as tf

In [12]:
tf.__version__

'2.1.0'

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [14]:
### Vocabulary size
voc_size=5000

## Onehot Representation

In [15]:
messages=X.copy()

In [16]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [17]:
messages.reset_index(inplace=True)

In [18]:
import nltk
import re
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Anoop
[nltk_data]     Mishra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    ##print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [21]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [22]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[3058, 733, 4629, 3259, 2199, 117, 4476, 2680, 299, 1065],
 [2806, 2356, 3007, 1278, 4526, 4309, 1306],
 [4518, 4034, 1374, 4913],
 [2572, 4269, 150, 3109, 314, 547],
 [2102, 4526, 2149, 1470, 3942, 933, 4526, 1918, 622, 1034],
 [2354,
  2317,
  2248,
  3640,
  758,
  2308,
  1943,
  3425,
  4505,
  1185,
  4905,
  3923,
  3179,
  137,
  1306],
 [442, 2805, 2591, 2993, 3623, 1865, 3689, 1664, 4311, 3302, 3911],
 [1892, 1542, 3169, 1147, 2417, 3637, 2308, 165, 4311, 3302, 3911],
 [1342, 574, 3372, 892, 2839, 2703, 3016, 4631, 2308, 2562],
 [4431, 1623, 703, 2592, 2242, 927, 3721, 1555],
 [1453, 110, 1509, 1711, 1073, 2211, 522, 1694, 1758, 3256, 1597],
 [3109, 3197, 2199, 2703, 2308, 2417],
 [3733, 4097, 2130, 2428, 1346, 397, 4007, 3555, 1707],
 [3431, 2257, 3865, 1272, 1246, 1031, 4343, 4311, 3302, 3911],
 [2625, 3106, 2683, 137, 4270, 4311, 3302, 3911],
 [394, 1529, 4974, 4213, 3142, 3022, 3573, 2302, 13, 2098],
 [2897, 2235, 2356],
 [3968, 666, 2446, 3814, 2308, 4757, 3919, 1306],


## Embedding Representation

In [23]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2680  299 1065]
 [   0    0    0 ... 4526 4309 1306]
 [   0    0    0 ... 4034 1374 4913]
 ...
 [   0    0    0 ... 4311 3302 3911]
 [   0    0    0 ... 2554 2345 3066]
 [   0    0    0 ... 4777 1423  702]]


In [24]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3058,
        733, 4629, 3259, 2199,  117, 4476, 2680,  299, 1065])

In [25]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
len(embedded_docs),y.shape

(18285, (18285,))

In [27]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [28]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

## Model Training

In [30]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 6s 493us/sample - loss: 0.3388 - accuracy: 0.8411 - val_loss: 0.2004 - val_accuracy: 0.9132
Epoch 2/10
12250/12250 [==============================] - 4s 310us/sample - loss: 0.1378 - accuracy: 0.9473 - val_loss: 0.1913 - val_accuracy: 0.9221
Epoch 3/10
12250/12250 [==============================] - 4s 312us/sample - loss: 0.0897 - accuracy: 0.9667 - val_loss: 0.2180 - val_accuracy: 0.9206
Epoch 4/10
12250/12250 [==============================] - 4s 366us/sample - loss: 0.0527 - accuracy: 0.9820 - val_loss: 0.2515 - val_accuracy: 0.9118
Epoch 5/10
12250/12250 [==============================] - 5s 380us/sample - loss: 0.0317 - accuracy: 0.9896 - val_loss: 0.2939 - val_accuracy: 0.9163
Epoch 6/10
12250/12250 [==============================] - 4s 361us/sample - loss: 0.0176 - accuracy: 0.9949 - val_loss: 0.3828 - val_accuracy: 0.9092
Epoch 7/10
12250/12250 [===========================

## Adding Dropout

In [31]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

## Performance Metrics And Accuracy

In [32]:
y_pred=model.predict_classes(X_test)

In [33]:
from sklearn.metrics import confusion_matrix

In [34]:
confusion_matrix(y_test,y_pred)

array([[2363, 1056],
       [1832,  784]], dtype=int64)

In [35]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5214581607290804